In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
from pandas.plotting import autocorrelation_plot as auto_corr

# To plot
import matplotlib.pyplot as plt  
%matplotlib inline    
import matplotlib as mpl
import seaborn as sns

#For date-time
import math
from datetime import datetime
from datetime import timedelta

# Another imports if needs
import itertools
import statsmodels.api as sm
import statsmodels.tsa.api as smt
import statsmodels.formula.api as smf

from sklearn.model_selection import train_test_split
from statsmodels.tsa.seasonal import seasonal_decompose as season
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.metrics import accuracy_score, balanced_accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.linear_model import LinearRegression 
from sklearn import preprocessing

from statsmodels.tsa.holtwinters import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller, acf, pacf
from statsmodels.tsa.arima_model import ARIMA
!pip install pmdarima
from pmdarima.utils import decomposed_plot
from pmdarima.arima import decompose
from pmdarima import auto_arima

import warnings
warnings.filterwarnings("ignore")

In [ ]:
pd.options.display.max_columns=100 # to see columns

In [ ]:
data_store = pd.read_csv('/kaggle/input/walmart-sales-forecast/stores.csv') #store data


In [ ]:
data_train = pd.read_csv('/kaggle/input/walmart-sales-forecast/train.csv') # train set


In [ ]:
data_features = pd.read_csv('/kaggle/input/walmart-sales-forecast/features.csv') #external information

# Merging Dataframes

In [ ]:
data_store.head()

In [ ]:
data_train.head()

In [ ]:
data_features.head()

In [ ]:
data = data_features.merge(data_train, on = ["Store","Date"], how = "inner").merge(data_store, on = ["Store"], how = "inner")

In [ ]:
data.head(10)

In [ ]:
IsHoliday_y = data["IsHoliday_y"]
IsHoliday_y.head(100)

In [ ]:
data.drop(['IsHoliday_y'], axis=1,inplace=True) # removing dublicated column

In [ ]:
data.rename(columns = {'IsHoliday_x':'IsHoliday'},inplace='True')

In [ ]:
data

In [ ]:
data.shape

In [ ]:
data['Store'].nunique()

In [ ]:
data['Dept'].nunique()

In [ ]:
weekly_sales_of_store_dept = pd.pivot_table(data, index='Store',columns='Dept', values='Weekly_Sales',aggfunc=np.mean)
weekly_sales_of_store_dept

In [ ]:
data.loc[data['Weekly_Sales']<=0]

In [ ]:
data = data.loc[data['Weekly_Sales'] > 0]

In [ ]:
data.shape # new data shape

In [ ]:
data['Date'].head(5).append(data['Date'].tail(5))

In [ ]:
sns.barplot(x='IsHoliday', y='Weekly_Sales', data=data)

In [ ]:
data_holiday = data.loc[data['IsHoliday']==True]
data_holiday['Date'].unique() 

In [ ]:
data_not_holiday = data.loc[data['IsHoliday']==False]
data_not_holiday['Date'].nunique() 

In [ ]:
# Super bowl dates in train set
data.loc[(data['Date'] == '2010-02-12')|(data['Date'] == '2011-02-11')|(data['Date'] == '2012-02-10'),'Super_Bowl'] = True
data.loc[(data['Date'] != '2010-02-12')&(data['Date'] != '2011-02-11')&(data['Date'] != '2012-02-10'),'Super_Bowl'] = False

In [ ]:
data


In [ ]:
# Labor day dates in train set
data.loc[(data['Date'] == '2010-09-10')|(data['Date'] == '2011-09-09')|(data['Date'] == '2012-09-07'),'Labor_Day'] = True
data.loc[(data['Date'] != '2010-09-10')&(data['Date'] != '2011-09-09')&(data['Date'] != '2012-09-07'),'Labor_Day'] = False

In [ ]:
# Thanksgiving dates in train set
data.loc[(data['Date'] == '2010-11-26')|(data['Date'] == '2011-11-25'),'Thanksgiving'] = True
data.loc[(data['Date'] != '2010-11-26')&(data['Date'] != '2011-11-25'),'Thanksgiving'] = False

In [ ]:
#Christmas dates in train set
data.loc[(data['Date'] == '2010-12-31')|(data['Date'] == '2011-12-30'),'Christmas'] = True
data.loc[(data['Date'] != '2010-12-31')&(data['Date'] != '2011-12-30'),'Christmas'] = False

In [ ]:
sns.barplot(x='Christmas', y='Weekly_Sales', data=data) # Christmas holiday vs not-Christmas

In [ ]:
sns.barplot(x='Thanksgiving', y='Weekly_Sales', data=data) # Thanksgiving holiday vs not-thanksgiving

In [ ]:
sns.barplot(x='Super_Bowl', y='Weekly_Sales', data=data) # Super bowl holiday vs not-super bowl

In [ ]:
sns.barplot(x='Labor_Day', y='Weekly_Sales', data=data) # Labor day holiday vs not-labor day

# Type Effect on Holidays

In [ ]:
data.groupby(['Christmas','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Christmas 

In [ ]:
data.groupby(['Labor_Day','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Labor Day

In [ ]:
data.groupby(['Thanksgiving','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Thanksgiving

In [ ]:
data.groupby(['Super_Bowl','Type'])['Weekly_Sales'].mean()  # Avg weekly sales for types on Super Bowl

In [ ]:
data

In [ ]:
type_counts = data["Type"].value_counts()
total_count = len(data)
type_percentages = (type_counts / total_count) * 100
type_percentages

In [ ]:
my_data = [51.16, 38.74 , 10.11 ]  #percentages
my_labels = 'Type A','Type B', 'Type C' # labels
plt.pie(my_data,labels=my_labels,autopct='%1.1f%%', textprops={'fontsize': 15}) #plot pie type and bigger the labels
plt.axis('equal')
mpl.rcParams.update({'font.size': 20}) #bigger percentage labels

plt.show()

Half of the stores are belongs to Type A.

In [ ]:
data.groupby('IsHoliday')['Weekly_Sales'].mean()

In [ ]:
# Plotting avg wekkly sales according to holidays by types
plt.style.use('seaborn-poster')
labels = ['Thanksgiving', 'Super_Bowl', 'Labor_Day', 'Christmas']
A_means = [27397.77, 20612.75, 20004.26, 18310.16]
B_means = [18733.97, 12463.41, 12080.75, 11483.97]
C_means = [9696.56,10179.27,9893.45,8031.52]

x = np.arange(len(labels))  # the label locations
width = 0.25  # the width of the bars

fig, ax = plt.subplots(figsize=(16, 8))
rects1 = ax.bar(x - width, A_means, width, label='Type_A')
rects2 = ax.bar(x , B_means, width, label='Type_B')
rects3 = ax.bar(x + width, C_means, width, label='Type_C')

# Add some text for labels, title and custom x-axis tick labels, etc.
ax.set_ylabel('Weekly Avg Sales')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()


def autolabel(rects):
    """Attach a text label above each bar in *rects*, displaying its height."""
    for rect in rects:
        height = rect.get_height()
        ax.annotate('{}'.format(height),
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),  # 3 points vertical offset
                    textcoords="offset points",
                    ha='center', va='bottom')


autolabel(rects1)
autolabel(rects2)
autolabel(rects3)

plt.axhline(y=17094.30,color='r') # holidays avg
plt.axhline(y=15952.82,color='green') # not-holiday avg

fig.tight_layout()

plt.show()

In [ ]:
data.sort_values(by='Weekly_Sales',ascending=False).head(10)

Also, it is not surprise that top 5 highest weekly sales are belongs to Thanksgiving weeks.

# To See the Size - Type Relation

In [ ]:
data_store.groupby('Type').describe()['Size'].round(2) # See the Size-Type relation

In [ ]:
plt.figure(figsize=(8,6)) # To see the type-size relation
fig = sns.boxplot(x='Type', y='Size', data=data, showfliers=False)

# Markdown Columns

Walmart gave markdown columns to see the effect if markdowns on sales. When I check columns, there are many NaN values for markdowns. I decided to change them with 0, because if there is markdown in the row, it is shown with numbres. So, if I can write 0, it shows there is no markdown at that date.

In [ ]:
data.isna().sum()

In [ ]:
data = data.fillna(0) # filling null's with 0

In [ ]:
data.isna().sum() # last null check

In [ ]:
data.describe() # to see weird statistical things

Minimum value for weekly sales is 0.01. Most probably, this value is not true but I prefer not to change them now. Because, there are many departments and many stores. It takes too much time to check each department for each store (45 store for 81 departments). So, I take averages for EDA.

# Deeper Look in Sales